In [14]:
import pandas as pd
import yfinance as yf
import seaborn as sns

In [15]:
# function to fetch data from yahoo finance for 1 year
def FetchData(Tickers,Period="max",Interval='1d',Start=None,End=None):

    """
    Parameters:
        tickers : str, list             List of tickers to download
        period : str             Valid periods: 1d,5d,1mo,3mo,6mo,1y,2y,5y,10y,ytd,max Either Use period parameter or use start and end
        interval : str             Valid intervals: 1m,2m,5m,15m,30m,60m,90m,1h,1d,5d,1wk,1mo,3mo Intraday data cannot extend last 60 days
        start: str             Download start date string (YYYY-MM-DD) or _datetime. Default is 1900-01-01
        end: str             Download end date string (YYYY-MM-DD) or _datetime. Default is now
        group_by : str             Group by 'ticker' or 'column' (default)
    """
    Data_Selected = yf.download(Tickers,start=Start,end=End,period=Period,interval=Interval)
    return(Data_Selected)



In [16]:
a=FetchData('T',Period='1y')

[*********************100%***********************]  1 of 1 completed
